In [ ]:
import pandas as pd
from Utils.calc_szenarios import calculate_df

In [ ]:
windkraftanlage_options = ['Ein', 'Aus']
bhkw_options = ['Aus', 'Ein', 'Lastspitzen (mit Wärmepumpe)']
pv_options = ['25 kWp', '500 kWp', '1.000 kWp', '1.500 kWp']
waermepumpe_options = [
    'Aus', 'Luft/Wasser 200', 'Luft/Wasser 400',
    'Luft/Wasser 600', 'Luft/Wasser 800', 'Luft/Wasser 1000',
    'Sole/Wasser 200', 'Sole/Wasser 400', 'Sole/Wasser 600',
    'Sole/Wasser 800', 'Sole/Wasser 1000'
]
wetter_typ=['Jetzt Normal'
,'Jetzt Kalt'
,'Jetzt Warm'
,'zukunft Normal'
,'zukunft Kalt'
,'zukunft Warm']
gastherme_options = ['Normal weiter', 'Zwei-Stoff-Brenner']
waermebedarfssenkung_options = ['0%','10%', '20%', '30%', '40%','50%']
vorlauf_temp_options=[55,65,75]
Windkraftanlage_options=[]
Bhkw_options=[]
Pv_options=[]
Waermepumpe_options=[]
Gastherme_options=[]
Waermebedarfssenkung_options=[]
Jahr=[]
Wetter_typ=[]
Netzbezug=[]
Netzeinspeisung=[]
Gasbezug=[]
max_gas=[]
max_p_el=[]
Volllaststunden_bhkw=[]
Cop=[]
Stromproduktion=[]
Stromverbrauch_hp=[]
Vorlauf_temp_options=[]

for waermebedarfssenkung in waermebedarfssenkung_options[0:1]:
    df=pd.read_csv('Output_data/Eingangsdaten_simulation.csv', index_col=0)
    df.index=pd.date_range(start='01/01/2022', end='01/01/2023',freq='15min')[0:35040]
    for weather_typ in wetter_typ[2:3]:
        if weather_typ.startswith('Jetzt'):
            wetter_time='2015'
        else:
            wetter_time='2045'
        if weather_typ.endswith('l'):
            weather_type='a_'
        elif weather_typ.endswith('t'):
            weather_type='w_'
        elif weather_typ.endswith('m'):
            weather_type='s_'
        print('TRY_1_'+weather_type+wetter_time+'_15min.csv')
        weather=pd.read_csv('Input_data/Wetterdaten/TRY_1_'+weather_type+wetter_time+'_15min.csv', index_col=0)
        weather.index=pd.date_range(start='01/01/2022', end='01/01/2023',freq='15min')[0:35040]
        df['PV_Süd [W]']=weather['PV_Süd [W]']
        df['PV_West [W]']=weather['PV_West [W]']
        df['PV_Ost [W]']=weather['PV_Ost [W]']
        df['PV - Vorhanden [W]']=weather['PV - Vorhanden [W]']
        weather_day=weather.resample('D').mean()
        df['Temperatur Luft [°C]']=weather['temperature [degC]']
        for day in weather_day.index:
            df.loc[str(day.date()),'Tages-Durchschnittstemperatur [°C]']=df.loc[str(day.date()),'Temperatur Luft [°C]'].mean()
        df.loc[df['Tages-Durchschnittstemperatur [°C]']>12.1, 'Raumwämebedarf [W]']=0
        df.loc[df['Tages-Durchschnittstemperatur [°C]']<12.1, 'Raumwämebedarf [W]']=(20-df.loc[df['Tages-Durchschnittstemperatur [°C]']<12.1, 'Temperatur Luft [°C]'])*43739
        df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']+df['BHKW Strom [W]']*1.13
        if weather_typ=='Jetzt Normal':
            if waermebedarfssenkung=='10%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.9
            elif waermebedarfssenkung=='20%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.8
            elif waermebedarfssenkung=='30%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.7
            elif waermebedarfssenkung=='40%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.6
            elif waermebedarfssenkung=='50%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.5
        for bhkw_option in bhkw_options:
            for waermepumpe_option in waermepumpe_options:
                for gastherme_option in gastherme_options:         
                    for windkraftanlage_option in windkraftanlage_options:
                        for pv_option in pv_options:
                            if waermepumpe_option=='Aus':
                                p_el_max, E_gf, E_gs, E_gas,p_gas_max, volllaststunden_bhkw, cop, stromproduktion, stromverbrauch_hp = calculate_df(df, windkraftanlage_option, bhkw_option, pv_option, waermepumpe_option, gastherme_option, 0)
                                Windkraftanlage_options.append(windkraftanlage_option)
                                Bhkw_options.append(bhkw_option)
                                Pv_options.append(pv_option)
                                Waermepumpe_options.append(waermepumpe_option)
                                Gastherme_options.append(gastherme_option)
                                Waermebedarfssenkung_options.append(waermebedarfssenkung)
                                Vorlauf_temp_options.append(0)
                                Wetter_typ.append(weather_typ)
                                Netzbezug.append(E_gs)
                                Netzeinspeisung.append(E_gf)
                                Gasbezug.append(E_gas)
                                max_gas.append(p_gas_max)
                                max_p_el.append(p_el_max)
                                Volllaststunden_bhkw.append(volllaststunden_bhkw)
                                Cop.append(0)
                                Stromproduktion.append(stromproduktion)
                                Stromverbrauch_hp.append(stromverbrauch_hp)
                            else:
                                for vorlauf_temp in vorlauf_temp_options:
                                    p_el_max, E_gf, E_gs, E_gas,p_gas_max, volllaststunden_bhkw, cop, stromproduktion, stromverbrauch_hp = calculate_df(df, windkraftanlage_option, bhkw_option, pv_option, waermepumpe_option, gastherme_option, vorlauf_temp)
                                    Windkraftanlage_options.append(windkraftanlage_option)
                                    Bhkw_options.append(bhkw_option)
                                    Pv_options.append(pv_option)
                                    Waermepumpe_options.append(waermepumpe_option)
                                    Gastherme_options.append(gastherme_option)
                                    Waermebedarfssenkung_options.append(waermebedarfssenkung)
                                    Vorlauf_temp_options.append(vorlauf_temp)
                                    Wetter_typ.append(weather_typ)
                                    Netzbezug.append(E_gs)
                                    Netzeinspeisung.append(E_gf)
                                    Gasbezug.append(E_gas)
                                    max_gas.append(p_gas_max)
                                    max_p_el.append(p_el_max)
                                    Volllaststunden_bhkw.append(volllaststunden_bhkw)
                                    Cop.append(cop)
                                    Stromproduktion.append(stromproduktion)
                                    Stromverbrauch_hp.append(stromverbrauch_hp)
results_df=pd.DataFrame()
results_df['Windkraftanlage_options']=Windkraftanlage_options
results_df['Bhkw_options']=Bhkw_options
results_df['Pv_options']=Pv_options
results_df['Waermepumpe_options']=Waermepumpe_options
results_df['Gastherme_options']=Gastherme_options
results_df['Waermebedarfssenkung_options']=Waermebedarfssenkung_options
results_df['Vorlauf_temp_options']=Vorlauf_temp_options
results_df['Wetter_typ']=Wetter_typ
results_df['Netzbezug']=Netzbezug
results_df['Netzeinspeisung']=Netzeinspeisung
results_df['Gasbezug']=Gasbezug
results_df['max_gas']=max_gas
results_df['max_p_el']=max_p_el
results_df['Volllaststunden_bhkw']=Volllaststunden_bhkw
results_df['Cop']=Cop
results_df['Stromproduktion']=Stromproduktion
results_df['Stromverbrauch_hp']=Stromverbrauch_hp
results_df.to_csv('Output_data/results_szenarien_2.csv')

In [ ]:
windkraftanlage_options = ['Ein', 'Aus']
bhkw_options = ['Aus', 'Ein', 'Lastspitzen (mit Wärmepumpe)']
pv_options = ['25 kWp', '500 kWp', '1.000 kWp', '1.500 kWp']
waermepumpe_options = [
    'Aus', 'Luft/Wasser 200', 'Luft/Wasser 400',
    'Luft/Wasser 600', 'Luft/Wasser 800', 'Luft/Wasser 1000',
    'Sole/Wasser 200', 'Sole/Wasser 400', 'Sole/Wasser 600',
    'Sole/Wasser 800', 'Sole/Wasser 1000'
]
wetter_typ=['Jetzt Normal'
,'Jetzt Kalt'
,'Jetzt Warm'
,'zukunft Normal'
,'zukunft Kalt'
,'zukunft Warm']
gastherme_options = ['Normal weiter', 'Zwei-Stoff-Brenner']
waermebedarfssenkung_options = ['0%','10%', '20%', '30%', '40%','50%']
vorlauf_temp_options=[45]#vorlauf_temp_options=[55,65,75]
Windkraftanlage_options=[]
Bhkw_options=[]
Pv_options=[]
Waermepumpe_options=[]
Gastherme_options=[]
Waermebedarfssenkung_options=[]
Jahr=[]
Wetter_typ=[]
Netzbezug=[]
Netzeinspeisung=[]
Gasbezug=[]
max_gas=[]
max_p_el=[]
Volllaststunden_bhkw=[]
Cop=[]
Stromproduktion=[]
Stromverbrauch_hp=[]
Vorlauf_temp_options=[]

for waermebedarfssenkung in waermebedarfssenkung_options[0:1]:
    df=pd.read_csv('Output_data/Eingangsdaten_simulation.csv', index_col=0)
    df.index=pd.date_range(start='01/01/2022', end='01/01/2023',freq='15min')[0:35040]
    for weather_typ in wetter_typ[2:3]:
        if weather_typ.startswith('Jetzt'):
            wetter_time='2015'
        else:
            wetter_time='2045'
        if weather_typ.endswith('l'):
            weather_type='a_'
        elif weather_typ.endswith('t'):
            weather_type='w_'
        elif weather_typ.endswith('m'):
            weather_type='s_'
        print('TRY_1_'+weather_type+wetter_time+'_15min.csv')
        weather=pd.read_csv('Input_data/Wetterdaten/TRY_1_'+weather_type+wetter_time+'_15min.csv', index_col=0)
        weather.index=pd.date_range(start='01/01/2022', end='01/01/2023',freq='15min')[0:35040]
        df['PV_Süd [W]']=weather['PV_Süd [W]']
        df['PV_West [W]']=weather['PV_West [W]']
        df['PV_Ost [W]']=weather['PV_Ost [W]']
        df['PV - Vorhanden [W]']=weather['PV - Vorhanden [W]']
        weather_day=weather.resample('D').mean()
        df['Temperatur Luft [°C]']=weather['temperature [degC]']
        for day in weather_day.index:
            df.loc[str(day.date()),'Tages-Durchschnittstemperatur [°C]']=df.loc[str(day.date()),'Temperatur Luft [°C]'].mean()
        df.loc[df['Tages-Durchschnittstemperatur [°C]']>12.1, 'Raumwämebedarf [W]']=0
        df.loc[df['Tages-Durchschnittstemperatur [°C]']<12.1, 'Raumwämebedarf [W]']=(20-df.loc[df['Tages-Durchschnittstemperatur [°C]']<12.1, 'Temperatur Luft [°C]'])*43739
        df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']+df['BHKW Strom [W]']*1.13
        if weather_typ=='Jetzt Normal':
            if waermebedarfssenkung=='10%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.9
            elif waermebedarfssenkung=='20%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.8
            elif waermebedarfssenkung=='30%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.7
            elif waermebedarfssenkung=='40%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.6
            elif waermebedarfssenkung=='50%':
                df['Raumwämebedarf [W]']=df['Raumwämebedarf [W]']*0.5
        for bhkw_option in bhkw_options:
            for waermepumpe_option in waermepumpe_options:
                for gastherme_option in gastherme_options:         
                    for windkraftanlage_option in windkraftanlage_options:
                        for pv_option in pv_options:
                            if waermepumpe_option=='Aus':
                                p_el_max, E_gf, E_gs, E_gas,p_gas_max, volllaststunden_bhkw, cop, stromproduktion, stromverbrauch_hp = calculate_df(df, windkraftanlage_option, bhkw_option, pv_option, waermepumpe_option, gastherme_option, 0)
                                Windkraftanlage_options.append(windkraftanlage_option)
                                Bhkw_options.append(bhkw_option)
                                Pv_options.append(pv_option)
                                Waermepumpe_options.append(waermepumpe_option)
                                Gastherme_options.append(gastherme_option)
                                Waermebedarfssenkung_options.append(waermebedarfssenkung)
                                Vorlauf_temp_options.append(0)
                                Wetter_typ.append(weather_typ)
                                Netzbezug.append(E_gs)
                                Netzeinspeisung.append(E_gf)
                                Gasbezug.append(E_gas)
                                max_gas.append(p_gas_max)
                                max_p_el.append(p_el_max)
                                Volllaststunden_bhkw.append(volllaststunden_bhkw)
                                Cop.append(0)
                                Stromproduktion.append(stromproduktion)
                                Stromverbrauch_hp.append(stromverbrauch_hp)
                            else:
                                for vorlauf_temp in vorlauf_temp_options:
                                    p_el_max, E_gf, E_gs, E_gas,p_gas_max, volllaststunden_bhkw, cop, stromproduktion, stromverbrauch_hp = calculate_df(df, windkraftanlage_option, bhkw_option, pv_option, waermepumpe_option, gastherme_option, vorlauf_temp)
                                    Windkraftanlage_options.append(windkraftanlage_option)
                                    Bhkw_options.append(bhkw_option)
                                    Pv_options.append(pv_option)
                                    Waermepumpe_options.append(waermepumpe_option)
                                    Gastherme_options.append(gastherme_option)
                                    Waermebedarfssenkung_options.append(waermebedarfssenkung)
                                    Vorlauf_temp_options.append(vorlauf_temp)
                                    Wetter_typ.append(weather_typ)
                                    Netzbezug.append(E_gs)
                                    Netzeinspeisung.append(E_gf)
                                    Gasbezug.append(E_gas)
                                    max_gas.append(p_gas_max)
                                    max_p_el.append(p_el_max)
                                    Volllaststunden_bhkw.append(volllaststunden_bhkw)
                                    Cop.append(cop)
                                    Stromproduktion.append(stromproduktion)
                                    Stromverbrauch_hp.append(stromverbrauch_hp)
results_df=pd.DataFrame()
results_df['Windkraftanlage_options']=Windkraftanlage_options
results_df['Bhkw_options']=Bhkw_options
results_df['Pv_options']=Pv_options
results_df['Waermepumpe_options']=Waermepumpe_options
results_df['Gastherme_options']=Gastherme_options
results_df['Waermebedarfssenkung_options']=Waermebedarfssenkung_options
results_df['Vorlauf_temp_options']=Vorlauf_temp_options
results_df['Wetter_typ']=Wetter_typ
results_df['Netzbezug']=Netzbezug
results_df['Netzeinspeisung']=Netzeinspeisung
results_df['Gasbezug']=Gasbezug
results_df['max_gas']=max_gas
results_df['max_p_el']=max_p_el
results_df['Volllaststunden_bhkw']=Volllaststunden_bhkw
results_df['Cop']=Cop
results_df['Stromproduktion']=Stromproduktion
results_df['Stromverbrauch_hp']=Stromverbrauch_hp
results_df.to_csv('Output_data/results_szenarien_8.csv')